In [1]:
%load_ext autoreload
%autoreload 2

%load_ext rpy2.ipython

d:\it\coding\github projects\my-university\iii курс\(python) эконометрика\venv\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [2]:
from data import DATA_FRAME

In [3]:
DATA_FRAME

,y,x1,x2,x3
0,195.197059,168.756262,31.972479,46.746935
1,195.190125,168.663856,32.073664,46.010348
2,196.225667,170.136834,33.630488,46.477245
3,197.097511,170.662084,34.285954,46.660334
4,193.652921,167.539127,32.767683,47.977245
...,...,...,...,...
295,195.360906,169.267803,33.383489,45.590076
296,194.890106,168.369894,32.911007,47.821680
297,195.509979,170.538656,35.594192,48.438330
298,195.875307,170.125282,34.178079,46.502980
